In [ ]:
from collections import namedtuple
import pathlib

In [ ]:
import numpy as np
import pandas as pd
import scipy
import toolz.curried as toolz

In [ ]:
import orchid

In [ ]:
# project_names = ('bakken', 'permian', 'montney')
# project_filenames = ('frankNstein_Bakken_UTM13_FEET.ifrac', 
#                      'Project_frankNstein_Permian_UTM13_FEET.ifrac', 
#                      'Project-frankNstein_Montney_UTM13_METERS.ifrac')
project_names = ('bakken', 'montney')
project_filenames = ('frankNstein_Bakken_UTM13_FEET.ifrac', 
                     'Project-frankNstein_Montney_UTM13_METERS.ifrac')
projects = toolz.pipe(
    dict(zip(project_names, project_filenames)),
    toolz.valmap(lambda fn: str(pathlib.Path(r'c:\src\Orchid.IntegrationTestData').joinpath(fn))),
    toolz.valmap(toolz.curry(orchid.load_project)))
projects

In [ ]:
# bakken_project, permian_project, montney_project = toolz.map(lambda pn: projects[pn], projects)
bakken_project, montney_project = toolz.map(lambda pn: projects[pn], projects)

In [ ]:
wells = toolz.pipe(projects,
    toolz.valmap(lambda project: project.wells),
    toolz.valmap(lambda well_map: [(w.name, w) for w in well_map]),
    toolz.valmap(dict))
wells

In [ ]:
def stages(project_name, well_name):
    return toolz.pipe(wells,
                      toolz.get_in([project_name, well_name]),
                      lambda w: w.stages,
                      toolz.map(lambda stage: (stage.display_stage_number, stage)),
                      dict)

In [ ]:
StageDetails = namedtuple('StageDetails', ['stage_number', 'display_name_with_well', 'md_top', 'md_bottom', 'cluster_count'])

In [ ]:
def stage_details(project_name, stage_list, stage_number):
    project = projects[project_name]
    length_unit = project.unit(orchid.physical_quantity.PhysicalQuantity.LENGTH.name)
    stage = stage_list[stage_number]
    return StageDetails(stage_number, stage.display_name_with_well, 
                        stage.md_top(length_unit), stage.md_bottom(length_unit), stage.cluster_count)

In [ ]:
bakken_demo_1h_stage_details = toolz.partial(stage_details, 'bakken', stages('bakken', 'Demo_1H'))
bakken_demo_2h_stage_details = toolz.partial(stage_details, 'bakken', stages('bakken', 'Demo_2H'))
bakken_demo_3h_stage_details = toolz.partial(stage_details, 'bakken', stages('bakken', 'Demo_3H'))
bakken_demo_4h_stage_details = toolz.partial(stage_details, 'bakken', stages('bakken', 'Demo_4H'))

In [ ]:
def print_stage_details(details_func, indices):
    for n in indices:
        print(details_func(n))

In [ ]:
def print_bakken_stages_details():
    for header, details_func, indices in [('Bakken Demo_1H', bakken_demo_1h_stage_details, [1, 50, 9, 33]),
                                          ('Bakken Demo_2H', bakken_demo_2h_stage_details, [1, 50, 21, 8]),
                                          ('Bakken Demo_3H', bakken_demo_3h_stage_details, [1]),
                                          ('Bakken Demo_4H', bakken_demo_4h_stage_details, [1, 35, 7, 26]),
                                         ]:
        print(f'\n{header}')
        print_stage_details(details_func, indices)
print_bakken_stages_details()

In [ ]:
montney_hori_01_stage_details = toolz.partial(stage_details, 'montney', stages('montney', 'Hori_01'))
montney_hori_02_stage_details = toolz.partial(stage_details, 'montney', stages('montney', 'Hori_02'))
montney_hori_03_stage_details = toolz.partial(stage_details, 'montney', stages('montney', 'Hori_03'))
montney_vert_01_stage_details = toolz.partial(stage_details, 'montney', stages('montney', 'Vert_01'))

In [ ]:
def print_montney_stages_details():
    for header, details_func, indices in [('Montney Hori_01', montney_hori_01_stage_details, [1, 15, 8, 2]),
                                          ('Montney Hori_02', montney_hori_02_stage_details, [1, 29, 8, 14]),
                                          ('Montney Hori_03', montney_hori_03_stage_details, [1, 28, 9, 20]),
                                          ('Montney Vert_01', montney_vert_01_stage_details, [1, 2, 3, 4]),
                                         ]:
        print(f'\n{header}')
        print_stage_details(details_func, indices)
print_montney_stages_details()